In [1]:
import os 
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import dates as mdates
import datetime as dt
import mplfinance as mpf
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import talib as ta

In [2]:
select_tf = ["day" , "5minute",'15minute',"60minute", "minute", "38minute" ]
tables = [] 
final_data = {}

for filename in tqdm(glob('TI_Data/*')):
    names = (filename.split(os.sep)[-1].split('_'))
    company_name = names[0]
    if names[1] in select_tf:
        data = pd.read_csv(filename, index_col=0)
        tf = names[1]
        variable_name = f"{company_name}_{tf}"
        locals()[variable_name] = data
        tables.append(variable_name)
        final_data[variable_name] = data

100%|████████████████████████████████████████████████████████████████████████████████| 606/606 [05:55<00:00,  1.71it/s]


In [3]:
def indexer(dicta):
    for df in dicta.values():
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Index_Column'}, inplace=True)
    return dicta

In [4]:
indexer(final_data)

{'ACC_15minute':             datec          date    close     high      low     open  volume  \
 0      2015-02-02  16468.156250  1541.85  1556.70  1536.05  1554.90   15738   
 1      2015-02-02  16468.166667  1542.35  1549.90  1540.50  1541.05   13350   
 2      2015-02-02  16468.177083  1546.30  1549.20  1541.05  1542.35    8198   
 3      2015-02-02  16468.187500  1545.25  1549.95  1544.80  1547.30    6736   
 4      2015-02-02  16468.197917  1541.60  1548.00  1540.35  1545.20    2590   
 ...           ...           ...      ...      ...      ...      ...     ...   
 43395  2022-10-11  19276.364583  2326.85  2339.50  2325.50  2336.55    5781   
 43396  2022-10-11  19276.375000  2328.45  2332.00  2323.25  2326.85    9073   
 43397  2022-10-11  19276.385417  2310.00  2329.50  2306.15  2328.45   18953   
 43398  2022-10-11  19276.395833  2310.20  2318.45  2305.70  2311.20   23999   
 43399  2022-10-11  19276.406250  2308.00  2316.70  2307.00  2311.05   37486   
 
        VolumePositive

In [5]:
data_day = {}
data_hour = {}
data_minute = {}
data_5minute = {}
data_15minute = {}
data_38minute = {}

for key, value in final_data.items():
    if key.endswith('_day'):
        data_day[key] = value
    elif key.endswith('_60minute'):
        data_hour[key] = value
    elif key.endswith('_minute'):
        data_minute[key] = value
    elif key.endswith('_5minute'):
        data_5minute[key] = value
    elif key.endswith('_15minute'):
        data_15minute[key] = value
    elif key.endswith('_38minute'):
        data_38minute[key] = value

In [6]:
class Trade:
    def __init__(self, *args, **kwargs):
        self.entry_flag = False
        self.entry_price = 0
        self.df = None
        self.profit_window = kwargs.get('profit_window', 3)
        self.loss_window = kwargs.get('loss_window', 1)

    def process_data(self, df):
        self.df = df.copy()  # Make a copy of the input DataFrame

        for i in range(len(self.df)):
            if i < 200:  # Adjust the skip condition to accommodate SMA_50
                self.df.loc[i, 'trade'] = 'n'
                continue
            elif (self.df.loc[i, 'SMA_20'] > self.df.loc[i, 'SMA_50'] and self.df.loc[i-1, 'SMA_20'] < self.df.loc[i, 'SMA_50'] or
                  self.df.loc[i, 'SMA_50'] > self.df.loc[i, 'SMA_200'] and self.df.loc[i-1, 'SMA_50'] < self.df.loc[i, 'SMA_200']) and not self.entry_flag:
                self.entry_price = self.df.loc[i, 'close']
                self.entry_flag = True
                self.df.loc[i, 'trade'] = 'E'  # Entry point marked with 'E'
            elif self.entry_flag:
                trade_value = ((self.df.loc[i, 'close'] - self.entry_price) / self.entry_price) * 100
                if trade_value >= self.profit_window:
                    self.df.loc[i, 'trade'] = 'P'
                    self.entry_flag = False
                elif trade_value <= -self.loss_window:
                    self.df.loc[i, 'trade'] = 'L'
                    self.entry_flag = False
                else:
                    self.df.loc[i, 'trade'] = 'C'
            else:
                self.df.loc[i, 'trade'] = 'C'

        return self.df

    def result_counts(self):
        entry_count = self.df[self.df['trade'] == 'E'].shape[0]
        profit_count = self.df[self.df['trade'] == 'P'].shape[0]
        loss_count = self.df[self.df['trade'] == 'L'].shape[0]
        
        total_trades = entry_count
        
        # Calculate percentage of profitable trades
        if total_trades > 0:
            percent_profitable = round((profit_count / total_trades) * 100, 2)
        else:
            percent_profitable = 0
        
        counts_array = [entry_count, profit_count, loss_count, percent_profitable
        ]
    
        return counts_array

In [9]:
for i in dict_list:
    results_df = pd.DataFrame()
    file = list(i.keys())[0].split('_')[1]
    print(f"File: {file}")

File: day
File: 60minute


In [10]:
# Assuming you have a list called profit_windows
results_df_list = [] 
# Assuming you have a list called profit_windows
profit_windows = [(1, 0.33), (3, 1), (15, 5), (30, 10)]

dict_list = [data_day, data_hour]
for i in dict_list:
    results_df = pd.DataFrame()
    file = list(i.keys())[0].split('_')[1]
    for key, df_day in tqdm(list(i.items())):
        df = df_day
        column_name = key.split('_')[0]
        for profit_window, loss_window in profit_windows:
            # Instantiate Trade class with the current profit_window and loss_window values
            trade_instance = Trade(profit_window=profit_window, loss_window=loss_window)
            
            # Process data using the Trade instance
            processed_data = trade_instance.process_data(df)
            
            # Get result counts using the result_counts method
            result_counts_array = trade_instance.result_counts()
            
            # Create a DataFrame from the current result
            current_result_df = pd.DataFrame({
                'Profit_Loss_Windows': [(profit_window, loss_window)],
                f'{column_name}': [result_counts_array]
            })
        
            # Append the current DataFrame to the list
            results_df_list.append(current_result_df)
    
    # Use pd.concat to concatenate the list of DataFrames
    results_df = pd.concat(results_df_list, ignore_index=True)
    
    # Display the final results DataFrame
    results_df
    grouped_df = results_df.groupby('Profit_Loss_Windows').agg(lambda x: pd.Series(x.dropna().values)).reset_index()
    grouped_df.to_csv('Results/MA/{file}.csv')

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [16:46<00:00,  9.97s/it]


## Day Run 

In [5]:
# Assuming you have a list called profit_windows
results_df_list = [] 
# Assuming you have a list called profit_windows
profit_windows = [(1, 0.33), (3, 1), (15, 5), (30, 10)]

for key, df_day in tqdm(list(data_day.items())):
    df = df_day
    column_name = key.split('_')[0]
    for profit_window, loss_window in profit_windows:
        # Instantiate Trade class with the current profit_window and loss_window values
        trade_instance = Trade(profit_window=profit_window, loss_window=loss_window)
        
        # Process data using the Trade instance
        processed_data = trade_instance.process_data(df)
        
        # Get result counts using the result_counts method
        result_counts_array = trade_instance.result_counts()
        
        # Create a DataFrame from the current result
        current_result_df = pd.DataFrame({
            'Profit_Loss_Windows': [(profit_window, loss_window)],
            f'{column_name}': [result_counts_array]
        })
    
        # Append the current DataFrame to the list
        results_df_list.append(current_result_df)

# Use pd.concat to concatenate the list of DataFrames
results_df = pd.concat(results_df_list, ignore_index=True)
    

NameError: name 'dict_list' is not defined

In [ ]:
grouped_df = results_df.groupby('Profit_Loss_Windows').agg(lambda x: pd.Series(x.dropna().values)).reset_index()

# Display the new DataFrame
grouped_df

In [ ]:
grouped_df.to_csv('Results/MA/day.csv')

## Hour Run 

In [ ]:
# Assuming you have a list called profit_windows
results_df_list = [] 
# Assuming you have a list called profit_windows
profit_windows = [(1, 0.33), (3, 1), (15, 5), (30, 10)]

for key, df_day in tqdm(list(data_hour.items())):
    df = df_day
    column_name = key.split('_')[0]
    for profit_window, loss_window in profit_windows:
        # Instantiate Trade class with the current profit_window and loss_window values
        trade_instance = Trade(profit_window=profit_window, loss_window=loss_window)
        
        # Process data using the Trade instance
        processed_data = trade_instance.process_data(df)
        
        # Get result counts using the result_counts method
        result_counts_array = trade_instance.result_counts()
        
        # Create a DataFrame from the current result
        current_result_df = pd.DataFrame({
            'Profit_Loss_Windows': [(profit_window, loss_window)],
            f'{column_name}': [result_counts_array]
        })
    
        # Append the current DataFrame to the list
        results_df_list.append(current_result_df)

# Use pd.concat to concatenate the list of DataFrames
results_df = pd.concat(results_df_list, ignore_index=True)

# Display the final results DataFrame
results_df

In [ ]:
grouped_df1 = results_df.groupby('Profit_Loss_Windows').agg(lambda x: pd.Series(x.dropna().values)).reset_index()

# Display the new DataFrame
grouped_df1

In [ ]:
grouped_df1.to_csv('Results/MA/hour.csv')

## 38 Minute_run

In [ ]:
# Assuming you have a list called profit_windows
results_df_list = [] 
# Assuming you have a list called profit_windows
profit_windows = [(1, 0.33), (3, 1), (15, 5), (30, 10)]

for key, df_day in tqdm(list(data_38minute.items())):
    df = df_day
    column_name = key.split('_')[0]
    for profit_window, loss_window in profit_windows:
        # Instantiate Trade class with the current profit_window and loss_window values
        trade_instance = Trade(profit_window=profit_window, loss_window=loss_window)
        
        # Process data using the Trade instance
        processed_data = trade_instance.process_data(df)
        
        # Get result counts using the result_counts method
        result_counts_array = trade_instance.result_counts()
        
        # Create a DataFrame from the current result
        current_result_df = pd.DataFrame({
            'Profit_Loss_Windows': [(profit_window, loss_window)],
            f'{column_name}': [result_counts_array]
        })
    
        # Append the current DataFrame to the list
        results_df_list.append(current_result_df)

# Use pd.concat to concatenate the list of DataFrames
results_df = pd.concat(results_df_list, ignore_index=True)

# Display the final results DataFrame
results_df

In [4]:

grouped_df =  pd.DataFrame()
grouped_df = results_df.groupby('Profit_Loss_Windows').agg(lambda x: pd.Series(x.dropna().values)).reset_index()

# Display the new DataFrame
grouped_df

NameError: name 'results_df' is not defined

## 15 Minute Run

In [ ]:
# Assuming you have a list called profit_windows
results_df_list = [] 
# Assuming you have a list called profit_windows
profit_windows = [(1, 0.33), (3, 1), (15, 5), (30, 10)]

for key, df_day in tqdm(list(data_15minunte.items())):
    df = df_day
    column_name = key.split('_')[0]
    for profit_window, loss_window in profit_windows:
        # Instantiate Trade class with the current profit_window and loss_window values
        trade_instance = Trade(profit_window=profit_window, loss_window=loss_window)
        
        # Process data using the Trade instance
        processed_data = trade_instance.process_data(df)
        
        # Get result counts using the result_counts method
        result_counts_array = trade_instance.result_counts()
        
        # Create a DataFrame from the current result
        current_result_df = pd.DataFrame({
            'Profit_Loss_Windows': [(profit_window, loss_window)],
            f'{column_name}': [result_counts_array]
        })
    
        # Append the current DataFrame to the list
        results_df_list.append(current_result_df)

# Use pd.concat to concatenate the list of DataFrames
results_df = pd.concat(results_df_list, ignore_index=True)

# Display the final results DataFrame
results_df

## Minute_run

In [ ]:
# Assuming you have a list called profit_windows
results_df_list = [] 
# Assuming you have a list called profit_windows
profit_windows = [(1, 0.33), (3, 1), (15, 5), (30, 10)]
for i in dict_list:
    results_df = 
    for key, df_day in tqdm(list(data_minute.items())):
        df = df_day
        column_name = key.split('_')[0]
        for profit_window, loss_window in profit_windows:
            # Instantiate Trade class with the current profit_window and loss_window values
            trade_instance = Trade(profit_window=profit_window, loss_window=loss_window)
            
            # Process data using the Trade instance
            processed_data = trade_instance.process_data(df)
            
            # Get result counts using the result_counts method
            result_counts_array = trade_instance.result_counts()
            
            # Create a DataFrame from the current result
            current_result_df = pd.DataFrame({
                'Profit_Loss_Windows': [(profit_window, loss_window)],
                f'{column_name}': [result_counts_array]
            })
        
            # Append the current DataFrame to the list
            results_df_list.append(current_result_df)
    
    # Use pd.concat to concatenate the list of DataFrames
    results_df = pd.concat(results_df_list, ignore_index=True)
    
    # Display the final results DataFrame
    results_df

In [ ]:
grouped_df2 = results_df.groupby('Profit_Loss_Windows').agg(lambda x: pd.Series(x.dropna().values)).reset_index()

# Display the new DataFrame
grouped_df2

In [ ]:
grouped_df1.to_csv('Results/MA/minute.csv')